In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
min_transactions = 200

N = 18 #number of pre-transactions used for predict
length = 20
num_of_items = 57 
NUL_ID = num_of_items - 1
time_slot = 24*60*60*1000 #1 day
time_slot

In [ ]:
RAWDATA_PATH = '/content/gdrive/MyDrive/NCKH/E-commerce/rawdata.csv'
HASH_FILE_PATH = '/content/gdrive/MyDrive/NCKH/E-commerce/hash_table.csv'
# NEW_DATA_PATH = '/content/gdrive/MyDrive/Timeseries/UserMoreThan200Trans.csv'
WEIGHTS_PATH = '/content/gdrive/MyDrive/NCKH/E-commerce/v5_attention.hdf5'

# Prepare data

Note: Ignore "Load data", "Ignore user" if you use processed data

## Load data, convert to dataframe

In [ ]:
df = pd.read_csv(RAWDATA_PATH)
df.head()

In [ ]:
hash_itemID = {}
idx = 0
for itemID in set(df.itemID):
  hash_itemID[itemID] = idx
  idx += 1

In [ ]:
with open(HASH_FILE_PATH,'w') as f:
  f.write('itemID,hash_value\n')
  for item in hash_itemID:
    f.write(f'{item},{hash_itemID[item]}\n')

In [ ]:
reverse_hash_itemID = {hash_itemID[i]:i for i in hash_itemID}

## Generate time series data

In [ ]:
import math

start_time = min(df.timestamp)
end_time = max(df.timestamp)
num_of_slots = math.ceil((end_time-start_time)/time_slot)
start_time,end_time,num_of_slots

### generate

In [ ]:
lst_user = list(set(df.userID))
len(lst_user)

In [ ]:
import more_itertools as mit
from google.colab import output
from functools import reduce
X_item2vec = []
X = []
y = []

is_exists = np.zeros(num_of_items,)

def onehot(x):
  #x: int
  lst = [0]*num_of_items
  lst[x] = 1
  return lst

def removeNul(x):
  #(1,2,-1,3,-1,2) -> (1,2,3,2)
  res = []
  return [str(i) for i in x if (i != -1)]

def flatten(x):
  #[[1],[2,1,3],[2]] -> [1,2,1,3,2]
  return [str(i) for c in x for i in c]

for userID in lst_user:
  subdf = df[df.userID == userID]

  # train_data for Item2Vec
  idx = ((subdf.timestamp - start_time) // time_slot).to_numpy()[:-1]
  itemIDs = list(map(lambda x: hash_itemID[x],subdf.itemID))[:-1]
  series = [[] for _ in range(num_of_slots)]
  for id,itemID in zip(idx,itemIDs):
    series[id].append(itemID)

  temp = list(mit.windowed(series, n=length, step=1))[:-1]  
  new_row = list(map(flatten,temp))
  X_item2vec.extend(new_row)


  # data for MLP
  history = list(map(lambda x: hash_itemID[x],subdf.itemID))
  new_x = list(mit.windowed(history, n=N, step=1))[:-1]
  X.extend(new_x)
  y.extend(list(map(onehot,history[N:])))

In [ ]:
from gensim.models import Word2Vec
#CBOW model
vecs = Word2Vec(X_item2vec, min_count=1, size=20, window=5, sg=0)

In [ ]:
vecs['31']

In [ ]:
vecs.similar_by_vector(vecs['31'])

#### split train-test

In [ ]:
def mapping(x):
  #x = (1,2,3) -> (vect(1),vect(2),vect(3))
  return [vecs[str(i)] for i in x]

In [ ]:
X = np.array(list(map(mapping,X)))
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
print(X_train[0],y_train[0],sep='\n')

In [ ]:
X_train.shape

# LSTM

In [ ]:
import numpy as np 
import os
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
import tensorflow as tf 


input_raw = Input((N,20)) #20 is length of embeded vector

lstm = LSTM(64, return_sequences=True, dropout=0.5)(input_raw) #50 is number of cells
lstm,state_h,state_c = LSTM(64, return_sequences=True, return_state=True, dropout=0.5)(lstm)

score = Dot(-1)([lstm,state_h])
softmax = Softmax()(score)
sm_reshape = Reshape(target_shape=(-1,1,))(softmax)
mul = Multiply()([lstm,sm_reshape])
context = Lambda(lambda x: K.sum(x, axis=-1), output_shape=lambda s: (s[0], s[1]))(mul)
concat = Concatenate()([context, state_h])
attention = Dense(20, activation='tanh')(concat)
attention = Dropout(0.5)(attention)


y_pred = Dense(num_of_items, activation='softmax')(attention)

model = Model(inputs = input_raw, outputs = y_pred)
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])


from keras.callbacks import EarlyStopping, ModelCheckpoint
model_checkpoint = ModelCheckpoint(WEIGHTS_PATH, monitor='loss', save_best_only=True, save_feq='epoch')
early_stop = EarlyStopping(verbose=True, patience=10, monitor='loss')
model.fit(X_train,y_train,batch_size=128,epochs=1000,verbose=True, callbacks=[early_stop,model_checkpoint])

Epoch 1/1000
435/435 [==============================] - 15s 14ms/step - loss: 3.6432 - accuracy: 0.1225
Epoch 2/1000
435/435 [==============================] - 6s 14ms/step - loss: 3.1834 - accuracy: 0.2035
Epoch 3/1000
435/435 [==============================] - 6s 14ms/step - loss: 2.9910 - accuracy: 0.2294
Epoch 4/1000
435/435 [==============================] - 6s 14ms/step - loss: 2.8717 - accuracy: 0.2456
Epoch 5/1000
435/435 [==============================] - 6s 15ms/step - loss: 2.7968 - accuracy: 0.2583
Epoch 6/1000
435/435 [==============================] - 7s 16ms/step - loss: 2.7396 - accuracy: 0.2650
Epoch 7/1000
435/435 [==============================] - 7s 16ms/step - loss: 2.6891 - accuracy: 0.2726
Epoch 8/1000
435/435 [==============================] - 7s 16ms/step - loss: 2.6496 - accuracy: 0.2777
Epoch 9/1000
435/435 [==============================] - 7s 16ms/step - loss: 2.6189 - accuracy: 0.2823
Epoch 10/1000
435/435 [==============================] - 7s 15ms/step - 

In [ ]:
model.save_weights(WEIGHTS_PATH)

In [ ]:
def post_process(x):
  idx = x.argsort()[-1:-6:-1]
  # idx = np.where(x>0.1)[0]
  return list(zip(idx,x[idx]))

In [ ]:
y_preds = model.predict(X_test)
y_preds = np.array(list(map(post_process,y_preds)))
acc = 0
for y_true, y_pred in zip(y_test,y_preds):
  key = np.argmax(y_true)
  lst_ypred = [x[0] for x in y_pred]
  if key in lst_ypred: acc += 1

acc/len(y_test)

0.7897476804231915

In [ ]:
#wrapper
def process(x):
  #x: array/ndarray of N item

  # inp = np.array(list(map(lambda t: hash_itemID[t],x)))
  inp = np.array([vecs[str(hash_itemID[i])] for i in x])
  y_pred = model.predict(np.array([inp]))
  y_pred = post_process(y_pred[0])
  out = list(map(lambda t: (reverse_hash_itemID[t[0]],t[1]),y_pred))

  return out

## Infer

In [ ]:
process([101,106,7,101,101,106,7,101,101,106,7,101,101,106,7,101,101,106])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


[(101, 0.13728186),
 (7, 0.11118631),
 (65, 0.10759728),
 (61, 0.1075136),
 (26, 0.0788148)]